# Analysis Notebook:

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os    
from dotenv  import load_dotenv
from pathlib import Path
%matplotlib inline

## Importing Data:

##### **<span style=color:red>  Install yfinance:**

In [2]:
# !pip install yfinance
import yfinance as yf

In [3]:
# Main Financial Data:
yf_dollar_df = yf.download("DX=F", start="2017-01-01", end="2020-12-31")
yf_gold_df = yf.download("GC=F", start="2017-01-01", end="2020-12-31")
yf_bitcoin_df = yf.download("BTC-USD", start="2017-01-01", end="2020-12-31")
yf_sp500_df = yf.download("^GSPC", start="2017-01-01", end="2020-12-31")

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded


In [4]:
# Other Data: M2 US Money Supply:
csvpath = Path("Resources/M2.csv")
m2_df = pd.read_csv(csvpath, index_col="DATE", parse_dates=True, infer_datetime_format=True).sort_values("DATE")
m2_df.rename(columns={"M2" : "M2(billions)"}, inplace=True)

In [5]:
# Rename Columns & Copy Original DataFrames with Selected Columns:

selected_dollar_df = yf_dollar_df.copy()
selected_dollar_df.columns = [('dollar_'+ column) for column in selected_dollar_df.columns]
selected_dollar_df = selected_dollar_df.iloc[:,[-2,-1]]

selected_gold_df = yf_gold_df.copy()
selected_gold_df.columns = [('gold_'+ column) for column in selected_gold_df.columns]
selected_gold_df = selected_gold_df.iloc[:,[-2,-1]]

selected_bitcoin_df = yf_bitcoin_df.copy()
selected_bitcoin_df.columns = [('bitcoin_'+ column) for column in selected_bitcoin_df.columns]
selected_bitcoin_df = selected_bitcoin_df.iloc[:,[-2,-1]]

selected_sp500_df = yf_sp500_df.copy()
selected_sp500_df.columns = [('sp500_'+ column) for column in selected_sp500_df.columns]
selected_sp500_df = selected_sp500_df.iloc[:,[-2,-1]]

## Concatenating Selected DataFrames for Further  Analysis:
joined_df = pd.concat([selected_dollar_df, selected_gold_df, selected_bitcoin_df, selected_sp500_df], axis = 'columns', join='inner', sort=True)
print(joined_df.tail())

            dollar_Adj Close  dollar_Volume  gold_Adj Close  gold_Volume  \
Date                                                                       
2020-12-21            89.949          39253          1879.2          136   
2020-12-22            90.546          21871          1866.6          233   
2020-12-23            90.340          27553          1874.7           82   
2020-12-28            90.275          18455          1877.2           75   
2020-12-29            89.918          17249          1879.7          343   

            bitcoin_Adj Close  bitcoin_Volume  sp500_Adj Close  sp500_Volume  
Date                                                                          
2020-12-21           22803.08     45852713981          3694.92    4732160000  
2020-12-22           23783.03     44171632681          3687.26    4023940000  
2020-12-23           23241.35     51146161904          3690.01    3772630000  
2020-12-28           27084.81     49056742893          3735.36    352746

## Calculating Historical Daily Returns:

In [6]:
joined_returns_df = pd.DataFrame()
joined_returns_df['dollar_return'] = joined_df['dollar_Adj Close'].pct_change()
joined_returns_df['gold_return'] = joined_df['gold_Adj Close'].pct_change()
joined_returns_df['bitcoin_return'] = joined_df['bitcoin_Adj Close'].pct_change()
joined_returns_df['sp500_return'] = joined_df['sp500_Adj Close'].pct_change()
joined_returns_df.dropna(inplace=True)
joined_returns_df.head()

,dollar_return,gold_return,bitcoin_return,sp500_return
Date,,,,
2017-01-04,-0.004767,0.002930,0.106233,0.005722
2017-01-05,-0.011499,0.013662,-0.122410,-0.000771
2017-01-06,0.006708,-0.006612,-0.109712,0.003517
2017-01-09,-0.002935,0.009898,0.000698,-0.003549
2017-01-10,0.000854,0.000591,0.005372,0.000000


# If Economic Environment Under "Normal" (2017 - 2019):

In [7]:
# Portfolio Returns under "Normal" 2017-2019 Scenario:
joined_returns_normal = joined_returns_df['20170101':'20191231'].drop(columns='sp500_return')
joined_returns_normal.tail()

,dollar_return,gold_return,bitcoin_return
Date,,,
2019-12-23,-0.000401,0.005289,0.018952
2019-12-26,-0.001234,0.018078,-0.015860
2019-12-27,-0.005859,0.002982,0.007062
2019-12-30,-0.001440,0.000462,0.000399
2019-12-31,-0.003620,0.003301,-0.013630


### Method A: Finding "Optimal" Weighting based on Trials for Different Weighting: 

In [8]:
# Set Number of Trials:
num_trials = 500

# Set Empty Array to Hold Values:
trial_weights_normal = np.zeros((num_trials, 3))  # np.zeros(shape) # 3 for three assets
trial_returns_normal = np.zeros(num_trials)
trial_std_normal = np.zeros(num_trials)
trial_sharpe_normal = np.zeros(num_trials)

#### Portfolio Standard Deviation Formula:
![Cov](Resources/images/cov.png)

In [9]:
# Trials for Different Random Weights:
for trial in range(num_trials):

    # Create Random Weights
    random_weights = np.array(np.random.random(3))

    # Rebalance Weights
    random_weights = random_weights / np.sum(random_weights)
    
    # Save Weights
    trial_weights_normal[trial,:] = random_weights

    # Expected Return
    trial_returns_normal[trial] = np.sum((joined_returns_normal.mean() * random_weights) *252)

    # Expected Std
    trial_std_normal[trial] = np.sqrt(np.dot(random_weights.T, np.dot(joined_returns_normal.cov() * 252, random_weights)))

    # Sharpe Ratio
    trial_sharpe_normal[trial] = trial_returns_normal[trial] / trial_std_normal[trial]

In [10]:
# Calculating Returns & Risk for the Specific Trail which gives the Highest Sharp Ratio:

# Finding the indices of the maximum shape value along an axis:
index_of_max_sharpe_normal = trial_sharpe_normal.argmax()  
max_weight_normal = trial_weights_normal[index_of_max_sharpe_normal, :]

max_return_normal = trial_returns_normal[index_of_max_sharpe_normal]
max_std_normal = trial_std_normal[index_of_max_sharpe_normal]
max_sharpe_normal = max_return_normal / max_std_normal

print(f"Best Trial Weight in Dollar Futures: {max_weight_normal[0]:.1%}")
print(f"Best Trial Weight in Gold Futures: {max_weight_normal[1]:.1%}")
print(f"Best Trial Weight in Bitcoin: {max_weight_normal[2]:.1%}")
print("-------------------------------------------------")
print(f"Best Trial Portfolio Return: {max_return_normal:0.3}")
print(f"Best Trial Portfolio Standard Deviation/Volatility: {max_std_normal:0.3}")
print(f"Best Trial Portfolio Sharpe: {max_sharpe_normal:0.5}")

Best Trial Weight in Dollar Futures: 9.2%
Best Trial Weight in Gold Futures: 75.8%
Best Trial Weight in Bitcoin: 15.0%
-------------------------------------------------
Best Trial Portfolio Return: 0.218
Best Trial Portfolio Standard Deviation/Volatility: 0.149
Best Trial Portfolio Sharpe: 1.4687


### Method B: Mathematical Optimization of the Portfolio:

In [11]:
import scipy
from scipy.optimize import minimize


In [12]:
# Define a Function which Returns the Portfolio returns, standard deviation, and sharpe:
def cal_returns_std_sharpe(weights):
    weights = np.array(weights)
    returns = np.sum(joined_returns_normal.mean()*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(joined_returns_normal.cov() * 252, weights)))
    sharpe_ratios = returns/std
    return np.array([returns, std, sharpe_ratios])

In [13]:
# Maximizing a Positive Sharpe is SAME as Minimizing a Negative Sharpe:
# So the Objective Function to be Minimized:
def objective_func(weights):
    return cal_returns_std_sharpe(weights)[2]*-1

In [14]:
#  Initial Guess ("x0") for Weighting (if equally distributed):
guess = [0.25, 0.25, 0.25]

In [15]:
# Constrians for Weighting:
# Sequence of (min, max) pairs for each element in x. None is used to specify no bound.
bounds = ((0, 1), (0, 1), (0, 1))

In [16]:
# The function defining the constraint that the total weight is one.
# Return 0 if sum of weights is 1.0
def check_weight(weights):
    return np.sum(weights)-1

# Constraints for COBYLA, SLSQP are defined as a list of dictionaries:
# ‘eq’ for equality, ‘ineq’ for inequality.
cons = ({'type':'eq', 'fun':check_weight})

In [17]:
# Minimize a scalar function of one or more variables using Sequential Least Squares Programming (SLSQP).
# scipy.optimize.minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
optimzation_results_normal = minimize(objective_func, guess, method='SLSQP', bounds=bounds, constraints=cons)
optimzation_results_normal

     fun: -1.4696771626167744
     jac: array([-6.70254230e-05,  6.03497028e-06,  1.44094229e-05])
 message: 'Optimization terminated successfully'
    nfev: 41
     nit: 10
    njev: 10
  status: 0
 success: True
       x: array([0.09841605, 0.76263742, 0.13894653])

In [18]:
# Print Results:
optimal_weights_normal = optimzation_results_normal["x"]
result_normal = cal_returns_std_sharpe(optimal_weights_normal)
print(f"Optimized Portfolio Weight in Dollar Futures: {optimal_weights_normal[0]:.1%}")
print(f"Optimized Portfolio Weight in Gold Futures: {optimal_weights_normal[1]:.1%}")
print(f"Optimized Portfolio Weight in Bitcoin: {optimal_weights_normal[2]:.1%}")
print("-------------------------------------------------")
print(f"Optimized Portfolio Return: {result_normal[0]:0.3}")
print(f"Optimized Portfolio Standard Deviation/Volatility: {result_normal[1]:0.3}")
print(f"Optimized Portfolio Sharpe Ratio: {result_normal[2]:0.5}")

Optimized Portfolio Weight in Dollar Futures: 9.8%
Optimized Portfolio Weight in Gold Futures: 76.3%
Optimized Portfolio Weight in Bitcoin: 13.9%
-------------------------------------------------
Optimized Portfolio Return: 0.208
Optimized Portfolio Standard Deviation/Volatility: 0.141
Optimized Portfolio Sharpe Ratio: 1.4697


### Plotting the Trials and the Best Trial:

In [19]:
import hvplot.pandas 
import holoviews as hv

In [20]:
trail_scatter = hv.Scatter((trial_std_normal, trial_returns_normal, trial_sharpe_normal), 
                           'Volatility', ['Return', 'Sharpe Ratio']).opts(
                            color='Sharpe Ratio', cmap='plasma', width=800, height=500, colorbar=True, padding=0.2
                            )

trail_max_sharpe = hv.Scatter([(max_std_normal, max_return_normal)]).opts(
    title='Trials & the Best Trial (Blue Dot) Under "Normal" (2017 - 2019):', active_tools=['wheel_zoom'], 
    fontscale=1.1, color='blue', line_color='black', size=10
    )

trail_scatter * trail_max_sharpe

:Overlay
   .Scatter.I  :Scatter   [Volatility]   (Return,Sharpe Ratio)
   .Scatter.II :Scatter   [x]   (y)

### Plotting the Efficent Frontier:
The efficient frontier is the set of optimal portfolios that offers the highest expected return for a defined level of risk <br> or the lowest risk for a given level of expected return. 

In [21]:
# Creat Evenly Spaced Numbers over a Specified Interval:
frontier_returns_normal = np.linspace(0, 1, 100)

In [22]:
# Define a Empty List to Hold Results:
frontier_volatility_normal = []

In [23]:
# Define an Object Function that Gives Minimum Volatility:
def minimize_volatility(weights):
    return cal_returns_std_sharpe(weights)[1]

In [24]:
# Calculte Minimul Volatility for Given Possible Returns:
for possible_return in frontier_returns_normal:
    
    # Adding Given Possible Returns as a Constrain:
    cons2 = ({'type':'eq', 'fun':check_weight},
            {'type':'eq','fun': lambda wight: cal_returns_std_sharpe(wight)[0] - possible_return}
           )
    
    result_normal2 = minimize(minimize_volatility, guess, method='SLSQP', bounds=bounds, constraints=cons2)
    
    # Store Calculation Results of Objective Function 'fun'-- Minimized Volatility
    frontier_volatility_normal.append(result_normal2['fun'])

In [25]:
# Plotting the Optimal Portolio:
opt_sharpe_normal = hv.Scatter([(result_normal[1], result_normal[0])]).opts(color='red', line_color='black', size=10)


# Plotting the Efficient Frontier:
efficient_frontier_normal = trail_scatter * opt_sharpe_normal * \
hv.Curve((frontier_volatility_normal, frontier_returns_normal)).opts(
    title='Efficient Frontier & the Optimal Porfolio (Red Dot) Under "Normal" (2017 - 2019):', 
    active_tools=['wheel_zoom'], fontscale=1.1, color='green', line_dash='dashed', axiswise=True
    )

efficient_frontier_normal

:Overlay
   .Scatter.I  :Scatter   [Volatility]   (Return,Sharpe Ratio)
   .Scatter.II :Scatter   [x]   (y)
   .Curve.I    :Curve   [x]   (y)

# If Economic Environment Under "Epidemic" (2020):

In [26]:
# Portfolio Returns under "Epidemic (2020)" Scenario:
joined_returns_epi = joined_returns_df['20200101':'20201231'].drop(columns='sp500_return')
joined_returns_epi.tail()

,dollar_return,gold_return,bitcoin_return
Date,,,
2020-12-21,-0.000044,-0.003447,-0.014473
2020-12-22,0.006637,-0.006705,0.042974
2020-12-23,-0.002275,0.004339,-0.022776
2020-12-28,-0.000720,0.001334,0.165372
2020-12-29,-0.003955,0.001332,0.010250


### Method A: Finding "Optimal" Weighting based on Trials for Different Weighting: 

In [27]:
# Set Number of Trials:
num_trials = 500

# Set Empty Array to Hold Values:
trial_weights_epi = np.zeros((num_trials, 3))  # np.zeros(shape) # 3 for three assets
trial_returns_epi = np.zeros(num_trials)
trial_std_epi = np.zeros(num_trials)
trial_sharpe_epi = np.zeros(num_trials)

In [28]:
# Trials for Different Random Weights:
for trial in range(num_trials):

    # Create Random Weights
    random_weights = np.array(np.random.random(3))

    # Rebalance Weights
    random_weights = random_weights / np.sum(random_weights)
    
    # Save Weights
    trial_weights_epi[trial,:] = random_weights

    # Expected Return
    trial_returns_epi[trial] = np.sum((joined_returns_epi.mean() * random_weights) *252)

    # Expected Std
    trial_std_epi[trial] = np.sqrt(np.dot(random_weights.T, np.dot(joined_returns_epi.cov() * 252, random_weights)))

    # Sharpe Ratio
    trial_sharpe_epi[trial] = trial_returns_epi[trial] / trial_std_epi[trial]

In [29]:
# Calculating Returns & Risk for the Specific Trail which gives the Highest Sharp Ratio:

# Finding the indices of the maximum shape value along an axis:
index_of_max_sharpe_epi = trial_sharpe_epi.argmax()  
max_weight_epi = trial_weights_epi[index_of_max_sharpe_epi, :]

max_return_epi = trial_returns_epi[index_of_max_sharpe_epi]
max_std_epi = trial_std_epi[index_of_max_sharpe_epi]
max_sharpe_epi = max_return_epi / max_std_epi

print(f"Best Trial Weight in Dollar Futures: {max_weight_epi[0]:.1%}")
print(f"Best Trial Weight in Gold Futures: {max_weight_epi[1]:.1%}")
print(f"Best Trial Weight in Bitcoin: {max_weight_epi[2]:.1%}")
print("-------------------------------------------------")
print(f"Best Trial Portfolio Return: {max_return_epi:0.3}")
print(f"Best Trial Portfolio Standard Deviation/Volatility: {max_std_epi:0.3}")
print(f"Best Trial Portfolio Sharpe: {max_sharpe_epi:0.5}")

Best Trial Weight in Dollar Futures: 1.2%
Best Trial Weight in Gold Futures: 43.6%
Best Trial Weight in Bitcoin: 55.2%
-------------------------------------------------
Best Trial Portfolio Return: 1.0
Best Trial Portfolio Standard Deviation/Volatility: 0.429
Best Trial Portfolio Sharpe: 2.336


### Method B: Mathematical Optimization of the Portfolio:

In [30]:
# Define a Function which Returns the Portfolio returns, standard deviation, and sharpe:
def cal_returns_std_sharpe(weights):
    weights = np.array(weights)
    returns = np.sum(joined_returns_epi.mean()*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(joined_returns_epi.cov() * 252, weights)))
    sharpe_ratios = returns/std
    return np.array([returns, std, sharpe_ratios])

In [31]:
# Maximizing a Positive Sharpe is SAME as Minimizing a Negative Sharpe:
# So the Objective Function to be Minimized:
def objective_func(weights):
    return cal_returns_std_sharpe(weights)[2]*-1

In [32]:
#  Initial Guess ("x0") for Weighting (if equally distributed):
guess = [0.25, 0.25, 0.25]

In [33]:
# Constrians for Weighting:
# Sequence of (min, max) pairs for each element in x. None is used to specify no bound.
bounds = ((0, 1), (0, 1), (0, 1))

In [34]:
# The function defining the constraint that the total weight is one.
# Return 0 if sum of weights is 1.0
def check_weight(weights):
    return np.sum(weights)-1

# Constraints for COBYLA, SLSQP are defined as a list of dictionaries:
# ‘eq’ for equality, ‘ineq’ for inequality.
cons = ({'type':'eq', 'fun':check_weight})

In [35]:
# Minimize a scalar function of one or more variables using Sequential Least Squares Programming (SLSQP).
# scipy.optimize.minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
optimzation_results_epi = minimize(objective_func, guess, method='SLSQP', bounds=bounds, constraints=cons)
optimzation_results_epi

     fun: -2.3372576897209063
     jac: array([ 0.09881675, -0.00023001,  0.00018203])
 message: 'Optimization terminated successfully'
    nfev: 24
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.        , 0.44174166, 0.55825834])

In [36]:
# Print Results:
optimal_weights_epi = optimzation_results_epi["x"]
result_epi = cal_returns_std_sharpe(optimal_weights_epi)
print(f"Optimized Portfolio Weight in Dollar Futures: {optimal_weights_epi[0]:.1%}")
print(f"Optimized Portfolio Weight in Gold Futures: {optimal_weights_epi[1]:.1%}")
print(f"Optimized Portfolio Weight in Bitcoin: {optimal_weights_epi[2]:.1%}")
print("-------------------------------------------------")
print(f"Optimized Portfolio Return: {result_epi[0]:0.3}")
print(f"Optimized Portfolio Standard Deviation/Volatility: {result_epi[1]:0.3}")
print(f"Optimized Portfolio Sharpe Ratio: {result_epi[2]:0.5}")

Optimized Portfolio Weight in Dollar Futures: 0.0%
Optimized Portfolio Weight in Gold Futures: 44.2%
Optimized Portfolio Weight in Bitcoin: 55.8%
-------------------------------------------------
Optimized Portfolio Return: 1.02
Optimized Portfolio Standard Deviation/Volatility: 0.434
Optimized Portfolio Sharpe Ratio: 2.3373


### Plotting the Trials and the Best Trial:

In [37]:
trail_scatter = hv.Scatter((trial_std_epi, trial_returns_epi, trial_sharpe_epi), 
                           'Volatility', ['Return', 'Sharpe Ratio']).opts(
                            color='Sharpe Ratio', cmap='plasma', width=800, height=500, colorbar=True, padding=0.2
                            )

trail_max_sharpe = hv.Scatter([(max_std_epi, max_return_epi)]).opts(
    title='Trials & the Best Trial (Blue Dot) Under  "Epidemic" (2020):', active_tools=['wheel_zoom'], 
    fontscale=1.1, color='blue', line_color='black', size=10
    )

trail_scatter * trail_max_sharpe

:Overlay
   .Scatter.I  :Scatter   [Volatility]   (Return,Sharpe Ratio)
   .Scatter.II :Scatter   [x]   (y)

### Plotting the Efficent Frontier:
The efficient frontier is the set of optimal portfolios that offers the highest expected return for a defined level of risk <br> or the lowest risk for a given level of expected return. 

In [38]:
# Creat Evenly Spaced Numbers over a Specified Interval:
frontier_returns_epi = np.linspace(0, 1.5, 100)

In [39]:
# Define a Empty List to Hold Results:
frontier_volatility_epi = []

In [40]:
# Define an Object Function that Gives Minimum Volatility:
def minimize_volatility(weights):
    return cal_returns_std_sharpe(weights)[1]

In [41]:
# Calculte Minimul Volatility for Given Possible Returns:
for possible_return in frontier_returns_epi:
    
    # Adding Given Possible Returns as a Constrain:
    cons2 = ({'type':'eq', 'fun':check_weight},
            {'type':'eq','fun': lambda wight: cal_returns_std_sharpe(wight)[0] - possible_return}
           )
    
    result_epi2 = minimize(minimize_volatility, guess, method='SLSQP', bounds=bounds, constraints=cons2)
    
    # Store Calculation Results of Objective Function 'fun'-- Minimized Volatility
    frontier_volatility_epi.append(result_epi2['fun'])

In [42]:
# Plotting the Optimal Portolio:
opt_sharpe_epi = hv.Scatter([(result_epi[1], result_epi[0])]).opts(color='red', line_color='black', size=10)


# Plotting the Efficient Frontier:
efficient_frontier_epi = trail_scatter * opt_sharpe_epi * \
hv.Curve((frontier_volatility_epi, frontier_returns_epi)).opts(
    title='Efficient Frontier & the Optimal Porfolio (Red Dot) Under  "Epidemic" (2020):', 
    active_tools=['wheel_zoom'], fontscale=1.1, color='green', line_dash='dashed', axiswise=True
    )

efficient_frontier_epi

:Overlay
   .Scatter.I  :Scatter   [Volatility]   (Return,Sharpe Ratio)
   .Scatter.II :Scatter   [x]   (y)
   .Curve.I    :Curve   [x]   (y)

# Table Summary:

In [43]:
opt_summary = pd.DataFrame({
    "Normal 2017-2019":[max_weight_normal[0],max_weight_normal[1],max_weight_normal[2],\
    max_return_normal,max_std_normal,max_sharpe_normal,optimal_weights_normal[0],\
    optimal_weights_normal[1],optimal_weights_normal[2],result_normal[0],result_normal[1],result_normal[2]],\
    
    "Epidemic 2020":[max_weight_epi[0],max_weight_epi[1],max_weight_epi[2],max_return_epi,max_std_epi,\
    max_sharpe_epi,optimal_weights_epi[0],optimal_weights_epi[1],optimal_weights_epi[2],result_epi[0],\
    result_epi[1],result_epi[2]]}, 
    
    index=["Best Trial Weight in Dollar Futures","Best Trial Weight in Gold Futures",\
            "Best Trial Weight in Bitcoin","Best Trial Portfolio Return","Best Trial Portfolio Standard Deviation/Volatility",\
            "Best Trial Portfolio Sharpe Ratio","Optimized Portfolio Weight in Dollar Futures","Optimized Portfolio Weight in Gold Futures",\
            "Optimized Portfolio Weight in Bitcoin","Optimized Portfolio Return","Optimized Portfolio Standard Deviation/Volatility",\
            "Optimized Portfolio Sharpe Ratio"])

opt_summary = opt_summary.iloc[6:,:].round(decimals=5)
opt_summary

,Normal 2017-2019,Epidemic 2020
Optimized Portfolio Weight in Dollar Futures,0.09842,0.00000
Optimized Portfolio Weight in Gold Futures,0.76264,0.44174
Optimized Portfolio Weight in Bitcoin,0.13895,0.55826
Optimized Portfolio Return,0.20774,1.01542
Optimized Portfolio Standard Deviation/Volatility,0.14135,0.43445
Optimized Portfolio Sharpe Ratio,1.46968,2.33726


# Candlestick with Volume:

In [44]:
from bokeh.models import BooleanFilter, CDSView, Select, Range1d, HoverTool, CrosshairTool
from bokeh.models.formatters import NumeralTickFormatter
from bokeh.layouts import gridplot
from bokeh.palettes import Category20
from bokeh.plotting import figure, output_file, show, ColumnDataSource

In [45]:
# Candlestick with Volume:
def plot_candlesticks(df_input):
    stock = ColumnDataSource(data=dict(index=[], Date=[], Open=[], Close=[], High=[], Low=[], Volume=[]))
    stock.data = stock.from_df(df_input.reset_index())    
   
    # Settings:
    VBAR_WIDTH = 0.5
    RED = Category20[7][6]
    BLUE = Category20[3][0]
    W_PLOT = 1500
    H_PLOT = 600
    
    # Tools Selections:
    TOOLS = "pan,xwheel_zoom,box_zoom,hover,crosshair,undo,redo,reset,save" 
    linked_crosshair = CrosshairTool(dimensions="both")

    # Graph One:
    p1 = figure(plot_width=W_PLOT, plot_height=H_PLOT, tools=TOOLS, active_scroll='xwheel_zoom', active_drag='pan',
               title="", toolbar_location='above')

    inc = stock.data['Close'] > stock.data['Open']
    dec = stock.data['Open'] > stock.data['Close']
    view_inc = CDSView(source=stock, filters=[BooleanFilter(inc)])
    view_dec = CDSView(source=stock, filters=[BooleanFilter(dec)])

    # Map dataframe indices to date strings and use as label overrides:
    p1.xaxis.major_label_overrides = {
        i+int(stock.data['index'][0]): date.strftime('%b-%d') for i, date in enumerate(pd.to_datetime(stock.data["Date"]))
    }
    p1.xaxis.bounds = (stock.data['index'][0], stock.data['index'][-1])

    # Rendering the Graph:
    p1.segment(x0='index', x1='index', y0='Low', y1='High', color=BLUE, source=stock, view=view_inc)
    p1.segment(x0='index', x1='index', y0='Low', y1='High', color=RED, source=stock, view=view_dec)

    p1.vbar(x='index', width=VBAR_WIDTH, top='Open', bottom='Close', fill_color=BLUE, line_color=BLUE,
           source=stock,view=view_inc, name="price")
    p1.vbar(x='index', width=VBAR_WIDTH, top='Open', bottom='Close', fill_color=RED, line_color=RED,
           source=stock,view=view_dec, name="price")

    # Formating Graph One:
    p1.xaxis.major_label_orientation = 3.1415/4
    p1.x_range.range_padding = 0.05
    p1.xaxis.ticker.desired_num_ticks = 50
    p1.yaxis.formatter = NumeralTickFormatter(format=' 0,0[.]000')
    p1.add_tools(linked_crosshair)
    
    # Select specific tool for the plot:
    price_hover = p1.select(dict(type=HoverTool))
    # Choose, which glyphs are active by glyph name
    price_hover.names = ["price"]
    # Creating tooltips
    price_hover.tooltips = [("Open", "@Open{0,0.00}"),
                            ("Close", "@Close{0,0.00}"),
                            ("Volume", "@Volume{(0.00 a)}")]
    price_hover.formatters = {"Date": 'datetime'}

    
    # Added-on Graph Two For Volume:
    p2 = figure(x_axis_type="datetime", tools="", toolbar_location=None, plot_width=W_PLOT, 
                plot_height=200, x_range=p1.x_range)
    
    # Map dataframe indices to date strings and use as label overrides:
    p2.xaxis.major_label_overrides = {
        i+int(stock.data['index'][0]): date.strftime('%b-%d') for i, date in enumerate(pd.to_datetime(stock.data["Date"]))
    }
    
    # Reder the Graph:
    p2.vbar(stock.data['index'], VBAR_WIDTH, stock.data['Volume'])
    
    # Formating Graph Two:
    p2.xaxis.major_label_orientation = 3.1415/4
    p2.xaxis.ticker.desired_num_ticks = 50
    p2.yaxis.formatter = NumeralTickFormatter(format='0,0[.]000')
    p2.add_tools(linked_crosshair)

    return gridplot([[p1],[p2]])

# Dashboard:

In [46]:
import panel as pn

In [47]:
# Create a Title for the Dashboard:
title = "Traditional Currency vs. Paper Currency vs. Crypto Currency, which one should we invest?"

# Create a Tab Layout for the Dashboard:
# Define a Welcome Text:
welcome_tab = pn.Column(
    "This dashboard presents a visual analysis of XXXXXXXXXXXXXXXXXX",
    "You can nagivate through the tabs above to explore more details about the XXXXXXXXXXXXXXX",
    "##**Dollar Futures:**", plot_candlesticks(yf_dollar_df),
    "##**Bitcoin: **", plot_candlesticks(yf_bitcoin_df),
    "##**Gold Futures:  **", plot_candlesticks(yf_gold_df),
    
)

optimization_tab = pn.Column(
        "### Portolio Optimazation and Efficient Frontiers:",
        opt_summary,
        pn.Row(efficient_frontier_normal,efficient_frontier_epi))

In [48]:
# Create a Dashboard:
dashboard = pn.Column(
    title,
    pn.Tabs(
        ("Welcome", welcome_tab),
        ("Portfolio Optimization", optimization_tab),
    )
)

In [49]:
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Markdown(str)
            [2] Markdown(str)
            [3] Bokeh(Column)
            [4] Markdown(str)
            [5] Bokeh(Column)
            [6] Markdown(str)
            [7] Bokeh(Column)
        [1] Column
            [0] Markdown(str)
            [1] DataFrame(DataFrame)
            [2] Row
                [0] HoloViews(Overlay)
                [1] HoloViews(Overlay)

In [50]:
dashboard.show()

Launching server at http://localhost:49549
